<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import urllib.parse as req

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''SELECT
                    COUNT(id)  -- считаем количество вакансий по id
                FROM
                    vacancies
            '''

In [5]:
# результат запроса
quantity_vacancies = pd.read_sql_query(query_3_1, connection)
quantity_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\505335110.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  quantity_vacancies = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''SELECT
                    COUNT(id)  -- считаем количество работодателей по id
                FROM
                    employers
            '''

In [7]:
# результат запроса
quantity_employers = pd.read_sql_query(query_3_2, connection)
quantity_employers

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\72524357.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  quantity_employers = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''SELECT
                    COUNT(id)  -- считаем количество регионов по id
                FROM
                    areas
            '''

In [9]:
# результат запроса
quantity_areas = pd.read_sql_query(query_3_3, connection)
quantity_areas

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\2486803467.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  quantity_areas = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''SELECT
                    COUNT(id)  -- считаем количество сфер деятельности по id
                FROM
                    industries
            '''

In [11]:
# результат запроса
quantity_industries = pd.read_sql_query(query_3_4, connection)
quantity_industries

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\3829655345.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  quantity_industries = pd.read_sql_query(query_3_4, connection)


,count
0,294


***

5. Дополните65льный анализ.

5.1. Запрос, который определит ТОП-25 работодателей с максимальным количеством вакансий.

In [12]:
# текст запроса
query_3_5_1 = f'''SELECT 
                    e.name,
                    COUNT(v.id) vacancies -- подсчет вакансий по их id для каждого работодателя 
                FROM 
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id -- соединяем таблицы vacancies и employers
                GROUP BY e.id -- группируем по id работодателя
                ORDER BY vacancies DESC -- сортировка по убыванию
                LIMIT 25 -- выводим первые 25 записей
             '''

In [13]:
# результат запроса
top_25_vacancies = pd.read_sql_query(query_3_5_1, connection)
top_25_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\1560675499.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_25_vacancies = pd.read_sql_query(query_3_5_1, connection)


,name,vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
5,ИК СИБИНТЕК,327
6,МТС,292
7,DataArt,247
8,Совкомбанк Технологии,204
9,Первый Бит,176


5.2. Запрос для определения количества работодателей с не менее, чем 10 вакансиями.

In [14]:
# текст запроса
query_3_5_2 = f'''WITH tab AS ( -- таблица с работодателями у каждого из которых не менее 10 вакансий
                    SELECT 
                        COUNT(*) -- подсчет вакансий для каждого работодателя
                    FROM 
                        vacancies v
                        LEFT JOIN employers e ON v.employer_id = e.id -- соединяем таблицы                 
                    GROUP BY v.employer_id -- группируем по id работодателя
                    HAVING COUNT(v.id) >= 10 -- не менее 10 вакансий у работодателя  
                )
                
                SELECT
                    COUNT(*) -- количество работодателей удовлетворяющих условию
                FROM
                    tab
             '''

In [15]:
# результат запроса
from_10_vacancies = pd.read_sql_query(query_3_5_2, connection)
from_10_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\2591236038.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  from_10_vacancies = pd.read_sql_query(query_3_5_2, connection)


,count
0,730


5.3. Запрос, который определит ТОП-50 регионов по количеству работодателей.

In [16]:
# текст запроса
query_3_5_3 = f'''SELECT 
                    a.name,
                    COUNT(e.id) employers -- подсчет работодателей по их id для каждого региона 
                FROM 
                    employers e
                    LEFT JOIN areas a ON e.area = a.id -- соединяем таблицы areas и employers
                GROUP BY a.id -- группируем по id региона
                ORDER BY employers DESC -- сортировка по убыванию количества работодателей
                LIMIT 50 -- выводим первые 50 строк
             '''

In [17]:
# результат запроса
top_50_areas_vacancies = pd.read_sql_query(query_3_5_3, connection)
top_50_areas_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\2014920760.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_50_areas_vacancies = pd.read_sql_query(query_3_5_3, connection)


,name,employers
0,Москва,5864
1,Санкт-Петербург,2217
2,Минск,1115
3,Алматы,721
4,Екатеринбург,609
5,Новосибирск,573
6,Казань,480
7,Нижний Новгород,426
8,Россия,410
9,Краснодар,409


5.4. Запрос, который определит ТОП-50 регионов по количеству вакансий.

In [18]:
# текст запроса
query_3_5_4 = f'''SELECT 
                    a.name,
                    COUNT(v.id) vacancies -- подсчет вакансий по их id для каждого из регионов 
                FROM 
                    vacancies v
                    LEFT JOIN areas a ON v.area_id = a.id -- соединяем таблицы areas и vacancies
                GROUP BY a.id -- группируем по id региона
                ORDER BY vacancies DESC -- сортировка по убыванию количества вакансий 
                LIMIT 50 -- выводим первые 50 значений
             '''

In [19]:
# результат запроса
top_50_areas_vacancies = pd.read_sql_query(query_3_5_4, connection)
top_50_areas_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\3385596592.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_50_areas_vacancies = pd.read_sql_query(query_3_5_4, connection)


,name,vacancies
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


***

выводы по предварительному анализу данных:
- Среднее число вакансий на всех работодателей: 2.1 (общее количество вакансий 49197, а общее количество работодателей 23501).
- среднее количество вакансий у работодателя с вакансиями: 3.3 (общее количество вакансий 49197, а количество работодателей с вакансиями 14906). 
- Максимальное количество вакансий 1933 у компании Яндекс.
- 25 компаний имеют количество вакансий от 100 и более (0.1 % от общего количества работодателей с вакансиями).
- На 17 месте по количеству вакансий находится ИП Поляков Денис Иванович, единственная компания в ТОП-25 не относящаяся к крупным компаниям.
- Количество компаний, которые разместили не менее 10 резюме - составляет 730. Можно считать, что это компании, которым требуется "существенное количество" IT специалистов.
- Распределение вакансий по регионам вполне ожидаемо - прямая корреляция с количеством населения, административным статусом и количеством экономических субъектов. В ТОП-50 по количеству вакансий входят административные центры субъектов Российской Федерации и крупнейшие города, а также столицы и крупные города стран СНГ.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [20]:
# текст запроса
query_4_1 = f'''SELECT
                    a.name,
                    COUNT(v.id) cnt  -- считаем количество вакансий по id
                FROM
                    vacancies v
                LEFT JOIN areas a ON v.area_id = a.id  -- соединяем таблицы vacancies и areas
                GROUP BY 1  -- группируем по названию региона
                ORDER BY 2 DESC  -- сортируем по количеству вакансий по убыванию
                LIMIT 5  -- выводим 5 первых значений
            '''

In [21]:
# результат запроса
top_vacancies = pd.read_sql_query(query_4_1, connection)
top_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\686296021.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_vacancies = pd.read_sql_query(query_4_1, connection)


,name,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [22]:
# текст запроса
query_4_2 = f'''SELECT
                    COUNT(id)  -- считаем количество по id
                FROM
                    vacancies
                WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL  -- проверяем что заполнено хотя бы одно поле с зарплатой
            '''

In [23]:
# результат запроса
salary_not_empty = pd.read_sql_query(query_4_2, connection)
salary_not_empty

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\4217971214.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_not_empty = pd.read_sql_query(query_4_2, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [24]:
# текст запроса
query_4_3 = f'''SELECT
                    ROUND(AVG(salary_from))::int salary_from,  -- находим среднее значение и округляем до целого
                    ROUND(AVG(salary_to))::int salary_to  -- для красоты приводим к типу int
                FROM
                    vacancies
            '''

In [25]:
# результат запроса
avg_salary = pd.read_sql_query(query_4_3, connection)
avg_salary

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\212845365.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary = pd.read_sql_query(query_4_3, connection)


,salary_from,salary_to
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [26]:
# текст запроса
query_4_4 = f'''SELECT
                    schedule,
                    employment,
                    COUNT(id)  -- подсчет количества вакансий
                FROM
                    vacancies
                GROUP BY 1, 2  -- группируем по графику, а внутри по типу трудоустройства
                ORDER BY 3 DESC  -- сортируем по количеству в порядке убывания
            '''

In [27]:
# результат запроса
sched_employ = pd.read_sql_query(query_4_4, connection)
sched_employ

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\1090706311.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sched_employ = pd.read_sql_query(query_4_4, connection)


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [28]:
# текст запроса
query_4_5 = f'''SELECT
                    experience,
                    COUNT(id)  -- подсчет количества вакансий
                FROM
                    vacancies
                GROUP BY 1  -- группируем по опыту работы
                ORDER BY 2  -- сортируем по количеству в порядке возрастания
            '''

In [29]:
# результат запроса
count_experience = pd.read_sql_query(query_4_5, connection)
count_experience

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\3463261295.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_experience = pd.read_sql_query(query_4_5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

6. Дополнительный анализ. 

6.1. Запрос для определения количества вакансий, у которых заполнены оба поля с зарплатой, либо не заполнены одно или оба поля с зарплатой. Дополнительно введем поле с процентным соотношением полученного количества.

In [30]:
# текст запроса
query_4_6_1 = f'''WITH salary_from_to AS ( -- таблица с 4 строками методом объединения 4 однострочных таблиц
                    SELECT
                        'Минимум & Максимум' salary,
                        COUNT(id) vacancies
                    FROM
                        vacancies 
                    WHERE
                        salary_from IS NOT NULL AND salary_to IS NOT NULL -- заполнены оба поля с зарплатой
                    UNION -- объединяем
                    SELECT
                        'Минимум',
                        COUNT(id)
                    FROM
                        vacancies 
                    WHERE
                        salary_from IS NOT NULL AND salary_to IS NULL -- заполнено только поле с минимумом зарплаты
                    UNION
                    SELECT
                        'Максимум',
                        COUNT(id)
                    FROM
                        vacancies 
                    WHERE
                        salary_from IS NULL AND salary_to IS NOT NULL -- заполнено только поле с максимумом зарплаты
                    UNION
                    SELECT
                        'Отсутствует',
                        COUNT(id)
                    FROM
                        vacancies 
                    WHERE
                        salary_from IS NULL AND salary_to IS NULL -- не заполнены оба поля с зарплатой 
                )
                SELECT -- итоговая таблица с добавленным столбцом "процент от общего количества вакансий"
                    salary,
                    vacancies,
                    ROUND(vacancies/(SELECT SUM(vacancies) FROM salary_from_to)*100)::int percent -- вычисление процента количества вакансий с приведением к целому
                FROM
                    salary_from_to
                ORDER BY vacancies DESC -- сортировка по убыванию количества вакансий
             '''

In [31]:
# результат запроса
salary_full_empty = pd.read_sql_query(query_4_6_1, connection)
salary_full_empty

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\1205354364.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary_full_empty = pd.read_sql_query(query_4_6_1, connection)


,salary,vacancies,percent
0,Отсутствует,25124,51
1,Минимум,11383,23
2,Минимум & Максимум,10102,21
3,Максимум,2588,5


***

выводы по детальному анализу вакансий:
- Распределение вакансий по регионам, как уже ранее показано, прямо коррелирует с количеством населения, административным статусом, количеством экономических субъектов в данном регионе. В ТОП-50 по количеству вакансий входят административные центры субъектов Российской Федерации, крупные города, а также столицы и крупные города стран СНГ. Так на долю Москвы приходится 5333 из 49197 вакансии, на доля Санкт-Петербурга 28751 из 49197. Минск на третьем месте 2112 из 49197.
- Более половины вакансий (51 %) не имеет каких-либо указаний на предлагаемую заработную плату. В 21 % имеется полная информация о предложении работодателя по зарплате. Минимум указан в 23 % вакансий. Максимум в 5 % вакансий.
- Средние значения для нижней и верхней границы зарплатной вилки составляют 71 и 110 тыс. рублей, соответственно. Дополнительно можно отметить, что уровень средних зарплат, полученный из этой базы данных не совсем соответствует реальным зарплатам в ИТ сфере в России на 2022 год. По этим данным, как пример, реальная медианная зарплата за первое полугодие 2022 года составила 140 тыс. рублей. Возможны две причины - зарплаты в вакансиях работодателей в-принципе ниже реальных рыночных, а также разные совокупности искомых сотрудников\работников и предоставивших информацию о своей заработной плате.
- Распределение вакансий для каждого сочетания типа рабочего графика и типа трудоустройства показывает, что порядка 80 % вакансий предполагают личное присутствие на рабочем месте (полный день, сменный и гибкий график). На долю удаленной работы приходится лишь порядка 19 %. Вахтовый метод где-то 0.8 %. Относительно формата занятости опять ожидаемо - более 93 % вакансий предполагают полную занятость. Частичная занятость и стажировка - всего чуть больше 4 и полутра процентов, соответственно. Проектная работа, менее 1 %.
- Требуемый опыт работы указан в подавляющем количестве вакансий - более 85 %. В 14 % вакансийе опыт не указан. Половину работодателей устраивает опыт от 1 до 3 лет. А почти 30 % опыт от 3 до 6 лет. А 2.7 % работодателей требуют больше 6 лет опыта.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [32]:
# текст запроса
query_5_1 = f'''SELECT
                    e.name,
                    COUNT(v.id)  -- считаем количество вакансий
                FROM
                    employers e
                LEFT JOIN vacancies v ON v.employer_id = e.id  -- соединяем таблицы vacancies и employers
                GROUP BY 1  -- группируем по имени работодателя
                ORDER BY 2 DESC  -- сортируем по количеству в порядке убывания
                LIMIT 5  -- выводим первые пять строк
            '''

In [33]:
# результат запроса
top_5_employer = pd.read_sql_query(query_5_1, connection)
top_5_employer

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\4042659804.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_5_employer = pd.read_sql_query(query_5_1, connection)


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [34]:
# текст запроса
# Для решения данного задания создадим дополнительно таблицы:
# - vacan с id регионов и количеством вакансий в каждом из них;
# - employ с id регионов и количеством работодателей в каждом из них.

query_5_2 = f'''SELECT
                    a.name,
                    vacan.vacan_quantity,
                    employ.employ_quantity
                FROM
                    areas a  -- присоединяем слева к таблице с регионами таблицы vacan и employ по ID регионов
                LEFT JOIN (SELECT
                               a.id id,
                               COUNT(v.id) vacan_quantity -- количество вакансий по регионам
                            FROM
                                vacancies v
                            LEFT JOIN areas a ON v.area_id = a.id
                            GROUP BY a.id  -- таблица vacan
                            ) vacan ON a.id = vacan.id
                            LEFT JOIN (SELECT
                                           a.id id,
                                           COUNT(e.id) employ_quantity -- количество работодателей по регионам
                                        FROM
                                            employers e
                                        LEFT JOIN areas a ON e.area = a.id
                                        GROUP BY a.id  -- таблица employ
                                        ) employ ON a.id = employ.id
                WHERE vacan_quantity IS NULL AND employ_quantity IS NOT NULL  -- оставим регионы, в которых отсутствуют вакансии и есть хотя бы один работодатель
                ORDER BY employ_quantity DESC  -- сортируем по количеству работодателей в порядке возрастания
                LIMIT 10  -- оставим первые 10 строк
            '''

In [35]:
# результат запроса
area_vacan_employ = pd.read_sql_query(query_5_2, connection)
area_vacan_employ

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\1732355076.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  area_vacan_employ = pd.read_sql_query(query_5_2, connection)


,name,vacan_quantity,employ_quantity
0,Россия,None,410
1,Казахстан,None,207
2,Московская область,None,75
3,Краснодарский край,None,19
4,Ростовская область,None,18
5,Беларусь,None,18
6,Азербайджан,None,17
7,Нижегородская область,None,16
8,Республика Татарстан,None,16
9,Узбекистан,None,15


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [36]:
# текст запроса
query_5_3 = f'''SELECT
                    e.name,
                    COUNT(DISTINCT v.area_id)  --количество уникальных значений регионов для каждого работодателя
                FROM
                    employers e
                LEFT JOIN vacancies v ON v.employer_id = e.id
                GROUP BY 1  -- группируем по имени работодателя
                ORDER BY 2 DESC  -- сортируем по количеству регионов в порядке убывания
            '''

In [37]:
# результат запроса
quant_area = pd.read_sql_query(query_5_3, connection)
quant_area

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\1425591209.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  quant_area = pd.read_sql_query(query_5_3, connection)


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [38]:
# текст запроса
query_5_4 = f'''SELECT
                    COUNT(*)
                FROM
                    employers e
                FULL JOIN employers_industries e_i ON e.id = e_i.employer_id  -- делаем полное соединение таблиц
                WHERE e_i.industry_id IS NULL  -- оставляем записи с пустыми id сферы деятельности
            '''

In [39]:
# результат запроса
empl_indus_empty = pd.read_sql_query(query_5_4, connection)
empl_indus_empty

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\654933434.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  empl_indus_empty = pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [40]:
# текст запроса
query_5_5 = f'''SELECT
                    e.name,
                    COUNT(e_i.industry_id)
                FROM
                    employers e
                JOIN employers_industries e_i ON e.id = e_i.employer_id  -- соединяем таблицы
                GROUP BY 1
                HAVING COUNT(e_i.industry_id) = 4  -- оставляем компании, у которых указано 4 сферы деятельности
                ORDER BY 1  -- сортируем по названию в алфавитном порядке
                OFFSET 2  -- обрезаем верхние 2 строки
                LIMIT 1  -- оставляем верхнюю из оставшихся, т.е. 3 в алфавитном списке
             '''

In [41]:
# результат запроса
empl_4_indus = pd.read_sql_query(query_5_5, connection)
empl_4_indus

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\1133966337.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  empl_4_indus = pd.read_sql_query(query_5_5, connection)


,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [42]:
# текст запроса
query_5_6 = f'''SELECT
                    COUNT(*)
                FROM
                    industries i
                JOIN employers_industries e_i ON e_i.industry_id = i.id  -- соединztv таблицs
                WHERE i.name = 'Разработка программного обеспечения'  -- оставляем записи, в которых сфера деятельности "Разработка программного обеспечения"
            '''

In [43]:
# результат запроса
soft_quantity = pd.read_sql_query(query_5_6, connection)
soft_quantity

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\3493476957.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  soft_quantity = pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [44]:
# код для получения списка городов-милионников
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
# преобразуем кириллицу в url
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
# достаем таблицу со страницы
df = pd.read_html(url)
# берем столбец с городами
df = df[1]
# преобразуем его в кортеж
list_citys = tuple(df['Город'])

In [45]:
# текст запроса
query_5_7 = f'''WITH tab AS (
                SELECT
                    a.name,
                    COUNT(*)::int cnt
                FROM
                    vacancies v
                LEFT JOIN areas a ON v.area_id = a.id  -- в таблицу вакансий добавляем названия работодателей 
                LEFT JOIN employers e ON v.employer_id = e.id  -- в таблицу вакансий добавляем названия регионов
                WHERE e.name = 'Яндекс'  -- оставляем записи, в которых работодатель "Яндекс"
                AND a.name IN {list_citys}  -- оставляем только регионы из списка регионов-миллионников 
                GROUP BY 1  -- группируем по названию регионов
                ORDER BY 2  -- сортируем по количеству вакансий в порядке возрастания
                )  -- cоздаем таблицу table с регионами из списка города-миллионники и количеством вакансий для каждого для Яндекса
                SELECT *
                FROM
                    tab
                UNION ALL  -- добавляем к таблице table общее количество вакансий по городам-миллионникам
                SELECT
                    'total',
                    SUM(cnt)  -- общая сумма вакансий
                FROM
                    tab
            '''

In [46]:
# результат запроса
millionaire_cities = pd.read_sql_query(query_5_7, connection)
millionaire_cities

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\1571217279.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  millionaire_cities = pd.read_sql_query(query_5_7, connection)


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

8. Дополнительный анализ

8.1. Запрос для определения, в каких регионах Яндекс публикует свои вакансии.

In [47]:
# текст запроса
query_5_8_1 = f'''SELECT
                    a.name,  -- выводим названия регионов
                    COUNT(v.id) quantity  -- количество вакансий по их id для каждого региона
                FROM
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id  -- соединяем таблицы vacancies и employers
                    LEFT JOIN areas a ON v.area_id = a.id  -- соединяем таблицы vacancies и areas
                WHERE e.name = 'Яндекс'  -- оставляем записи где работодатель Яндекс
                GROUP BY a.id  -- группировка по id региона
                ORDER BY quantity DESC  -- сортируем по количеству вакансий по убыванию                          
            '''

In [48]:
# результат запроса
yandex_areas = pd.read_sql_query(query_5_8_1, connection)
yandex_areas

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\2552198180.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yandex_areas = pd.read_sql_query(query_5_8_1, connection)


,name,quantity
0,Москва,54
1,Санкт-Петербург,42
2,Екатеринбург,39
3,Нижний Новгород,36
4,Новосибирск,35
...,...,...
176,Кызылорда,1
177,Томилино,1
178,Костанай,1
179,Актобе,1


***

выводы по анализу работодателей:
- Работодатели на первом и пятом месте по количеству вакансий: Яндекс и Газпромнефть.
- 186 регионов без вакансий. Это страны, и другие составные субъекты, отдельные части которых присутствуют в перечне регионов без текущей потребности в ИТ специалистах.
- Если посмотреть количество регионов для каждого работодателя, в которых он публикует свои вакансии, то можно увидеть корреляцию с количеством вакансий у работодателей. У кого много вакансий - у того много и регионов, в которых он публикует свои вакансии. У кого одна вакансия - тот опубликовал ее в одном регионе. Яндекс ожидаемо на первом месте c результатом 181 регион. Размещение вакансий: Москва, СПб, все административные центры субъектов РФ, самые крупные города, плюс тоже самое по Казахстану и Беларуси. Наблюдается полная корреляция с величиной и статусом региона, первые пять: Москва 54, Санкт-Петербург 42, Екатеринбург 39, Нижний Новгород 36, Новосибирск 35.
- Почти 8.5 тыс. работодателей не указали сферы деятельности в которых они предлагают работу.
- В отличие от предыдущих, компания 2ГИС указала 4 сферы деятельности.
- У 3553 работодателей указана такая сфера деятельности "Разработка программного обеспечения".
- Количество вакансий Яндекса по городам-миллионникам - 16. Москва первая в списке, СПб второй и т.д. Всего 485 вакансий, что составляет четверть от всех вакансий Яндекса.
- ИП Поляков Денис Иванович находится на 4 месте среди работодателей по количеству регионов опубликованных вакансий (88 регионов) Более подробную информацию об этом работодателе добудем в Дополнительных исследованиях к данному проекту.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [49]:
# текст запроса
query_6_1 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE lower(name) LIKE '%data%' OR lower(name) LIKE '%данн%'  -- переводим названия вакансий с нижний регистр и проверяем на содержание в них "data" или "данн"
            '''

In [50]:
# результат запроса
data_vacancies = pd.read_sql_query(query_6_1, connection)
data_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\2935617630.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_vacancies = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [51]:
# текст запроса
query_6_2 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE (name ~* 'data scientist' 
                OR name ~* 'data science' 
                OR name ~* 'исследователь данных' 
                OR (name ~* 'ML' AND name !~* 'HTML') 
                OR name ~* 'machine learning' 
                OR name ~* 'машинн.*обучен.*')  -- условие для выбора вакансий, которые соответствуют  вакансиям для дата-сайентистов
                AND
                (name ~* 'junior' OR
                experience = 'Нет опыта' OR
                employment = 'Стажировка')  --условие для выбора вакансий для специалистов уровня Junior
            '''

In [52]:
# результат запроса
jun_vacancies = pd.read_sql_query(query_6_2, connection)
jun_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\2311562088.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  jun_vacancies = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [53]:
# текст запроса
query_6_3 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (key_skills ~* 'SQL' OR key_skills ~* 'postgres')  --условие для выбора вакансий где в качестве ключевого навыка указан SQL или postgres
            '''

In [54]:
# результат запроса
sql_vacancies = pd.read_sql_query(query_6_3, connection)
sql_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\3698489625.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_vacancies = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [55]:
# текст запроса
query_6_4 = f'''SELECT
                    COUNT(*)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    key_skills ~* 'Python'  -- условие для выбора вакансий где в качестве ключевого навыка указан Python
            '''

In [56]:
# результат запроса
python_vacancies = pd.read_sql_query(query_6_4, connection)
python_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\100281186.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  python_vacancies = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [57]:
# текст запроса
# Количество ключевых навыков в вакансии будем рассчитывать по количеству разделителей "\t" + 1 в ячейке key_skills
query_6_5 = f'''SELECT
                    ROUND(AVG(length(key_skills) - length(replace(key_skills,'\t','')) + 1), 2)
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                '''

In [58]:
# результат запроса
avg_vacancies = pd.read_sql_query(query_6_5, connection)
avg_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\1487478692.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_vacancies = pd.read_sql_query(query_6_5, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [59]:
# текст запроса
# Среднюю зарплату будем считать так: ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2)) - данное выражение даст нам в итоге либо две минимальных зарплаты,
# либо две максимальных зарплаты, либо минимальную и максимальную. После деления на 2 получим либо среднюю, либо минимальную ну или максимальную.
query_6_6 = f'''SELECT
                    experience,
                    ROUND(AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2))::int avg
                FROM
                    vacancies
                WHERE
                    (name ~* 'data scientist' OR 
                    name ~* 'data science' OR 
                    name ~* 'исследователь данных' OR
                    (name ~ 'ML' AND name !~* 'HTML') OR
                    name ~* 'machine learning' OR
                    name ~* 'машинн.*обучен.*') -- применение условия для соответствия вакансии дата-сайентиста
                    AND
                    (salary_from IS NOT NULL OR salary_to IS NOT NULL) -- заполнено хотя бы одно из двух полей с зарплатой
                GROUP BY experience -- группируем по опыту
            '''

In [60]:
# результат запроса
avg_salary = pd.read_sql_query(query_6_6, connection)
avg_salary

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\4147299713.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary = pd.read_sql_query(query_6_6, connection)


,experience,avg
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


***

7. Дополнительный анализ

7.1. Запрос для определения работодателей, которым требуются дата-сайентисты.

In [61]:
# текст запроса
query_6_7_1 = f'''SELECT 
                    e.name,
                    COUNT(v.id) vacancies -- подсчет вакансий по их id для каждого работодателя 
                FROM 
                    vacancies v
                    LEFT JOIN employers e ON v.employer_id = e.id -- соединяем таблицы  employers и vacancies
                WHERE
                    (v.name ~* 'data scientist' OR 
                    v.name ~* 'data science' OR 
                    v.name ~* 'исследователь данных' OR
                    (v.name ~ 'ML' AND v.name !~* 'HTML') OR
                    v.name ~* 'machine learning' OR
                    v.name ~* 'машинн.*обучен.*') -- условие соответствия вакансии дата-сайентиста
                GROUP BY e.id -- группируем по id работодателя
                ORDER BY vacancies DESC -- сортировка по убыванию
             '''

In [62]:
# результат запроса
employers_ds = pd.read_sql_query(query_6_7_1, connection)
employers_ds

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\2535929142.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employers_ds = pd.read_sql_query(query_6_7_1, connection)


,name,vacancies
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
...,...,...
220,Федеральное автономное учреждение Государствен...,1
221,Представительство ООО «Квалитет» (Российская Ф...,1
222,DIGINETICA,1
223,Дром,1


***

выводы по предметному анализу:
- 1771 вакансия имеет отношение к данным (3.6 % от всего количества). Всего лишь!!!
- Всего лишь 51 вакансия для начинающего дата-сайентиста!!!
- Но Стоит овладеть навыками SQL или postgres и уже можно всерьез рассматривать 201 вакансию (речь о вакансиях для дата-сайентистов).
- Для тех кто знает Python открыта 351 вакансия (73 % всех вакансий для дата-сайентистов).
- 6.41 ключевых навыков в среднем указывают в вакансиях для дата-сайентистов.
- Имеется прямая корреляция с опытом, чем больше опыт - тем больший размер заработной платы предлагает работодатель. Сотрудникам без опыта готовы платить 75 тыс. рублей. С опытом от года и до 3 лет - 140 тыс. руб. А от 3 до 6 лет - 243 тыс. руб. Правда стоит отметить, что из 480 вакансий для дата-сайентистов заработная плата указана лишь в 62. А вот в 418 работодателей не указали размер зарплаты. Значит, строим предположение о корреляции на основании только лишь 13 % вакансий.
- 225 работодателям требуются дата-саентисты, а это 1.5 % от общего количества.

# Общий вывод по проекту

Общий анализ:

- Общее количество вакансий 49197, общее количество работодателей 23501, работодателей с вакансиями только 14906. Т.о. среднее количество вакансий у работодателя: 3.3 (рассматриваем работодателей с вакансиями).
- Распределение вакансий по регионам, прямо коррелирует с «масштабом региона». В ТОП-50 по количеству вакансий входят административные центры субъектов Российской Федерации и крупные города, а также столицы и крупные города стран СНГ. Так на долю Москвы приходится почти 11 % от общего количества вакансий, доля Санкт-Петербурга почти 6 %. На третьем месте Минск с более, чем 4 %.
= Более половины вакансий - 51 %, не имеют каких-либо указаний на предлагаемую заработную плату. В 21 % имеется полная информация о предложении работодателя по зарплате. Минимум указан в 23 % вакансий, а максимум в 5 % вакансий. Средние значения для нижней и верхней границы зарплатной вилки составляют 71 и 110 тыс. рублей, соответственно.
- Более 80 % вакансий предполагают личное присутствие на рабочем месте (полный день, сменный и гибкий график), на долю удаленной работы приходится около 19 %. Вахтовый метод примерно 0.8 %. Относительно формата занятости: более 93 % вакансий предполагают полную занятость. Частичная занятость и стажировка - всего чуть больше 4 и полутра процентов, соответственно. Проектная работа, менее 1 %.
- Требуемый опыт работы указан в подавляющем количестве вакансий - более 85 %. При этом половина устанавливает опыт от 1 до 3 лет, почти 30 % от 3 до 6 лет и 2.7 % - больше 6 лет опыта. Опыт прямо связан с зарплатными предложениями, при росте опыта растет предлагаемая зарплата.
- Если посмотреть количество регионов для каждого работодателя, в которых он публикует свои вакансии, то мы видим корреляцию с количеством вакансий у работодателей, т.е. при росте числа вакансий растет число регионов публикации.

Анализ вакансий, связанных с «Data Science»:

- 1771 вакансия имеет отношение к данным (в названии присутствуют или «data» или «данн»), то есть 3.6 % от всего количества вакансий. Если говорить о вакансиях для дата-сайентистов (исходя из перечня указанных условий для названий), то их 480 или чуть меньше 1 %.
- В 51 вакансии указан уровень Junior, это 10.6 % от всех вакансий, для дата-сайентистов.
- SQL или postgres в качестве ключевых навыков указаны в 201 вакансии для дата-сайентистов. Python присутствует в 351 вакансии, что составляет 73 % всех вакансий для дата-сайентистов.
- 6.41 среднее количество ключевых навыков в вакансиях для дата-сайентистов.
- Сотрудникам без опыта готовы платить 75 тыс. рублей, с опытом от года до 3 лет - 140 тыс. руб., а от 3 и до 6 лет - 243 тыс. руб. Для опыта свыше 6 лет – зарплата не указана. Основываясь на информации по всем вакансиям можно считать, что предлагаемая зарплата должна быть в 5-6 раз выше, чем для соискателей без опыта. Максимальная и минимальная указанные зарплаты: 55.5 и 550 тыс. руб.

Выводы вытекающие из анализа вакансий, связанных с «Data Science»:

Специальность «Data Science» достаточно редко востребована в общем количестве вакансий ИТ сферы. Вакансии предлагаются в основном крупными компаниями (банки, телекоммуникационные корпорации, интернет-гиганты, крупные промышленные компании и т.п. Востребованы специалисты с опытом, и с набором определенных ключевых навыков. Вакансии предлагаются в крупных городах. Возможная удаленная работа. Заработная плата в целом в два – три раза выше, чем в среднем для ИТ сферы. И зависит в первую очередь от опыта.

### Дополнительные исследования

7. Добудем информацию об ИП Поляков Денис Иванович, а именно:
1) из какого региона этот работодатель,
2) в каких сферах деятельности он предлагает вакансии,
3) в каких регионах предлагаются вакансии,
4) какие именно вакансии (название, график, оплата, регион вакансии и т.д.).

In [63]:
# текст запроса
query_7_1 = f'''SELECT 
                    e.id id_eployer,
                    e.name,
                    a.name area
                FROM 
                    employers e
                JOIN areas a ON e.area = a.id -- соединяем таблицы employers и areas
                WHERE e.id = 5130287 -- это id ИП Поляков Денис Иванович
            '''

In [64]:
# результат запроса
mr_polyakov = pd.read_sql_query(query_7_1, connection)
mr_polyakov

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\676973739.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mr_polyakov = pd.read_sql_query(query_7_1, connection)


,id_eployer,name,area
0,5130287,Поляков Денис Иванович,Владивосток


In [65]:
# текст запроса
query_7_2 = f'''SELECT 
                    COUNT(*) indusiries
                FROM 
                    employers_industries e_i
                LEFT JOIN industries i ON e_i.industry_id = i.id -- соединяем таблицы employers_industries и industries
                WHERE e_i.employer_id = 5130287 -- id ИП Поляков Денис Иванович
             '''   

In [66]:
# результат запроса
mr_polyakov_industries = pd.read_sql_query(query_7_2, connection)
mr_polyakov_industries

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\2403880047.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mr_polyakov_industries = pd.read_sql_query(query_7_2, connection)


,indusiries
0,0


In [67]:
# текст запроса
query_7_3 = f'''SELECT 
                    a.name area_vacancy,
                    COUNT(*) quantity
                FROM 
                    vacancies v
                LEFT JOIN areas a ON v.area_id = a.id -- соединяем таблицы vacancies и areas 
                WHERE v.employer_id = 5130287 -- id ИП Поляков Денис Иванович
                GROUP BY a.name -- группируем по названию региона
                ORDER BY quantity DESC, area_vacancy -- сортируем по количеству вакансий в порядке убывания и по названию региона по алфавиту 
             '''

In [68]:
# результат запроса
mr_polyakov_areas = pd.read_sql_query(query_7_3, connection)
mr_polyakov_areas

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\299267780.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mr_polyakov_areas = pd.read_sql_query(query_7_3, connection)


,area_vacancy,quantity
0,Орск,3
1,Балаково,2
2,Бийск,2
3,Братск,2
4,Брянск,2
...,...,...
83,Усолье-Сибирское,1
84,Уссурийск,1
85,Чита,1
86,Южно-Сахалинск,1


In [69]:
# текст запроса
query_7_4 = f'''SELECT 
                    v.name,
                    v.key_skills,
                    v.schedule,
                    v.experience,
                    v.employment,
                    v.salary_from,
                    v.salary_to,
                    a.name area_vacancy
                FROM 
                    vacancies v
                LEFT JOIN areas a ON v.area_id = a.id -- соединяем таблицы vacancies и areas
                WHERE v.employer_id = 5130287 -- id ИП Поляков Денис Иванович
                ORDER BY  v.salary_from DESC, v.salary_to DESC
             '''

In [70]:
# результат запроса
mr_polyakov_vacancies = pd.read_sql_query(query_7_4, connection)
mr_polyakov_vacancies

C:\Users\sysadmin\AppData\Local\Temp\ipykernel_21164\521560378.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mr_polyakov_vacancies = pd.read_sql_query(query_7_4, connection)


,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_vacancy
0,Компьютерный мастер,Навыки продаж\tНавыки работы с возражениями\tУ...,Полный день,От 1 года до 3 лет,Полная занятость,70000,NaN,Тула
1,Компьютерный мастер,None,Полный день,От 1 года до 3 лет,Полная занятость,60000,NaN,Кемерово
2,Компьютерный мастер,None,Полный день,От 1 года до 3 лет,Полная занятость,60000,NaN,Курск
3,Компьютерный мастер,None,Полный день,От 1 года до 3 лет,Полная занятость,60000,NaN,Оренбург
4,Компьютерный мастер,None,Полный день,От 1 года до 3 лет,Полная занятость,60000,NaN,Набережные Челны
...,...,...,...,...,...,...,...,...
122,Ученик компьютерного мастера,None,Полный день,Нет опыта,Стажировка,30000,50000.0,Геленджик
123,Ученик компьютерного мастера,None,Полный день,Нет опыта,Стажировка,30000,50000.0,Балаково
124,Ученик компьютерного мастера,None,Полный день,Нет опыта,Стажировка,30000,50000.0,Новороссийск
125,Ученик компьютерного мастера,None,Полный день,Нет опыта,Стажировка,30000,50000.0,Тюмень


Выводы по дополнительным исследованиям:

1. ИП Поляков Денис Иванович зарегистрирован во Владивостоке.

2. Cферs деятельности ИП Поляков Д.И. в данной базе данных не указаны.

3. ИП Поляков Д.И. разместил вакансии для 88 регионов.

4. Всего размещено 127 вакансий. Это либо компьютерный мастер, либо ученик компьютерного матера.

Предлагаемая зарплата:
- минимальная для ученика компьютерного мастера 30 тыс. руб., для компьютерного мастера от 40 тыс. руб.
- максимальная в 5 регионах 70 тыс. руб. (Ижевск, Новотроицк, Курск, Нижневартовск, Воткинск), в 8 регионах 55 тыс. руб., в остальных - 50 тыс. руб. Стоит отметить Стерлитамак где максимальная предлагаемая зарплата 150 тыс. руб.

Требование по опыту работы, либо отсутствует, либо от 1 до 3 лет, соответственно предлагаемой зарплате и вакансии.
Для ученика компьютерного мастера предлагается стажировка, для компьютерного мастера - полная занятость.
График для всех вакансий указан как полный день.
Требуемые навыки в основном не указаны. Исключение, 6 вакансий, а именно: компьютерный мастер в Архангельске, Тамбове, Орле, Томске, Симферополе и Туле. Там требуются следующие навыки: Техники продаж, Уверенный пользователь ПК, Ремонт ПК, Настройка ПО, Настройка ПК, Сборка ПК, Грамотная речь, Выездное обслуживание, Диагностика, Активные продажи, Ремонт электроники, Диагностика ПК, Опытный пользователь ПК.

Исходя из выше изложенного, скорее всего ИП Поляков Денис Иванович это федеральная сеть мастерских по обслуживанию и ремонту компьютерной техники, установке и настройке ПО.

In [72]:
# закрыdftv соединение после окончания работы
connection.close()